## Audio Post Processing Function

In [ ]:
import tensorflow as tf
import numpy as np
import librosa
from scipy import signal

### Filter Params

In [ ]:
# Bandpass to filter the noise
nyq = 0.5 * 16000
low_value = 1000
high_value = 7000
low_cutoff = low_value / nyq
high_cutoff = high_value / nyq

### Import IRs

***Note:*** Reset path to IRs

List of Mic IR
1. IR_Crystal.wav
2. IR_Lomo52A5M.wav
3. IR_OktavaMD57.wav
4. IR_AKGD12.wav
5. IR_GaumontKalee.wav
6. IR_STC4035.wav
7. IR_MelodiumRM6.wav

List of Speaker IR
1. IR_ClestionBD300.wav
2. IR_CelestionV30E606.wav
3. IR_JensenCab.wav
4. IR_Unknown.wav

List of Room IR
1. BRIR.wav

In [ ]:
# Speaker IR
ir_speaker_dir = '/beegfs/kp2218/test_runs/conv_test/data/audio/ir_speaker/IR_ClestionBD300.wav'
ir_speaker, fs_speaker = librosa.load(ir_speaker_dir, sr=16000, mono=True)

# Microphone IR
ir_mic_dir = '/beegfs/kp2218/test_runs/conv_test/data/audio/ir_mic/IR_GaumontKalee.wav'
ir_mic, fs_mic = librosa.load(ir_mic_dir, sr=16000, mono=True)

# Room IR
ir_room_dir = '/beegfs/kp2218/test_runs/conv_test/data/audio/ir_room/BRIR.wav'
ir_room, fs_room = librosa.load(ir_room_dir, sr=16000, mono=True)

### Post Processing function (Tensorflow)
##### Input: Audio tensor (input)
##### Output: Processed audio tensor (audio_out)

In [ ]:
def audio_post_processing(input_file):
    
    with tf.name_scope("post_processing") as scope:
        
        # Convolve with Speaker IR
        ir_speaker_tensor = tf.placeholder(dtype=tf.float32, shape=[None], name="Speaker_Tensor")
        ir_speaker_tensor = tf.expand_dims(ir_speaker_tensor, 1)
        ir_speaker_tensor = tf.expand_dims(ir_speaker_tensor, 2)
        speaker_out = tf.nn.conv1d(input_file, ir_speaker_tensor, 1, padding="SAME", name="Speaker_Out")
        
        # Add noise
        s = speaker_out.get_shape().as_list()
        noise_tensor = tf.random.normal([1,65536,1], mean=0, stddev=5e-3, dtype=tf.float32, name="Noise_param")
        
        # ** Add filter **
        
        speaker_out = tf.add(speaker_out, noise_tensor, name="Speaker_plus_Noise")

        # Convolve with Room IR
        ir_room_tensor = tf.placeholder(dtype=tf.float32, shape=[None], name="Room_Tensor")
        ir_room_tensor = tf.expand_dims(ir_room_tensor, 1)
        ir_room_tensor = tf.expand_dims(ir_room_tensor, 2)
        room_out = tf.nn.conv1d(speaker_out, ir_room_tensor, 1, padding="SAME", name="Room_Out")

        # Convolve with Mic IR
        ir_mic_tensor = tf.placeholder(dtype=tf.float32, shape=[None], name="Mic_Tensor")
        ir_mic_tensor = tf.expand_dims(ir_mic_tensor, 1)
        ir_mic_tensor = tf.expand_dims(ir_mic_tensor, 2)
        audio_out = tf.nn.conv1d(room_out, ir_mic_tensor, 1, padding="SAME", name="Audio_Out")

        return audio_out

### Post Processing function (Numpy)
##### Input: Audio array (input)
##### Output: Processed audio array (audio_out)

In [ ]:
def audio_post_processing(input_file):

    # Convolve with Speaker IR
    speaker_out = np.convolve(input_file, ir_speaker, mode='same')

    # Add noise
    noise_param = np.random.normal(0, 5e-3, size=speaker_out.shape)

    b, a = signal.cheby1(15, 4, [low_cutoff, high_cutoff], btype='bandpass')
    noise_param = signal.filtfilt(b, a, noise_param)

    speaker_out += noise_param

    # Convolve with Room IR
    room_out = np.convolve(speaker_out, ir_room, mode='same')

    # Convolve with Mic IR
    audio_out = np.convolve(room_out, ir_mic, mode='same')

    return audio_out